# Final Capstone Project
## Best location for a restaurant among top 12 cities worldwide

#### by Ignacio de Juan
#### November 2020, Final Project Assignment for IBM Data Science Course

### Introduction / Business Problem

We need to take the decision of where is the best location to open a new restaurant of a certain cuisine (e.g. Italian, Japanese, Spanish, etc). Which are the cities with a highest potential based on the level of concentration of certain cuisine and the demand of each cuisine based also on the assessment of the existing restaurants? If possible, provide with a recommendation for the location of the restaurant and the cuisine that would be likely to be more successful.

### Data

We will take data from 12 top cities worldwide.

The criteria to select those cities is to choose among cities that have a similar GDP per capita and a large enough base of restaurants so that the study is statistically significant.

We will be using Wikipedia to identify the target cities of our study and Foursquare as our database of venue data.

## Set up

In [1]:
import pandas as pd
import requests
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.style.use(['seaborn'])

In [2]:
# !pip install python-dotenv

# Credentials file
%load_ext dotenv
%dotenv

import os

import json

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests # library to handle requests
from pandas import json_normalize

import numpy as np

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
#Setting up Foursquare

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

## Data 

### Selecting the cities

We retrieve the list of worldwide cities ranked by GDP PPP

Source: Wikipedia, https://en.wikipedia.org/wiki/List_of_cities_by_GDP_(PPP)_per_capita

We obtain the venue data available in Foursquare.

We restrict venue type to restaurant

In [4]:
# Code to retrieve the cities

url = 'https://en.m.wikipedia.org/wiki/List_of_United_States_cities_by_population'
dfs = pd.read_html(url)

print(len(dfs))

print(dfs[3])

df = dfs[3]

11
     2019rank               City              State[c]  2019estimate  \
0           1   New York City[d]              New York       8336817   
1           2        Los Angeles            California       3979576   
2           3            Chicago              Illinois       2693976   
3           4         Houston[3]                 Texas       2320268   
4           5            Phoenix               Arizona       1680992   
5           6    Philadelphia[e]          Pennsylvania       1584064   
6           7        San Antonio                 Texas       1547253   
7           8          San Diego            California       1423851   
8           9             Dallas                 Texas       1343573   
9          10           San Jose            California       1021795   
10         11             Austin                 Texas        978908   
11         12    Jacksonville[f]               Florida        911507   
12         13         Fort Worth                 Texas       

In [5]:
df['City'] = df['City'].str.replace("\[.*\]","")
df.head()

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York City,New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston,Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


In [6]:
df = df.head(20)
df

,2019rank,City,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York City,New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston,Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W
5,6,Philadelphia,Pennsylvania,1584064,1526006,+3.80%,134.2 sq mi,347.6 km2,"11,683/sq mi","4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W
6,7,San Antonio,Texas,1547253,1327407,+16.56%,461.0 sq mi,"1,194.0 km2","3,238/sq mi","1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W
7,8,San Diego,California,1423851,1307402,+8.91%,325.2 sq mi,842.3 km2,"4,325/sq mi","1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°W
8,9,Dallas,Texas,1343573,1197816,+12.17%,340.9 sq mi,882.9 km2,"3,866/sq mi","1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W
9,10,San Jose,California,1021795,945942,+8.02%,177.5 sq mi,459.7 km2,"5,777/sq mi","2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°W


In [14]:
cities = list(df['City'])
print(cities)



['New York City', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose', 'Austin', 'Jacksonville', 'Fort Worth', 'Columbus', 'Charlotte', 'San Francisco', 'Indianapolis', 'Seattle', 'Denver', 'Washington']


In [15]:
# Foursquare API credentials and endpoint
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
VERSION = '20200616'
URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, 5000)

In [28]:
cuisines = {
    'Afghan': '503288ae91d4c4b30a586d67', 
    'African': '4bf58dd8d48988d1c8941735', 
    'American': '4bf58dd8d48988d14e941735', 
    'Burmese': '56aa371be4b08b9a8d573568', 
    'Cambodian': '52e81612bcbc57f1066b7a03', 
    'Chinese': '4bf58dd8d48988d145941735', 
    'Filipino': '4eb1bd1c3b7b55596b4a748f', 
    'Himalayan': '52e81612bcbc57f1066b79fb', 
    'Indonesian': '4deefc054765f83613cdba6f', 
    'Japanese': '4bf58dd8d48988d111941735', 
    'Korean': '4bf58dd8d48988d113941735', 
    'Malay': '4bf58dd8d48988d156941735', 
    'Mongolian': '4eb1d5724b900d56c88a45fe', 
    'Thai': '4bf58dd8d48988d149941735', 
    'Tibetan': '52af39fb3cf9994f4e043be9', 
    'Vietnamese': '4bf58dd8d48988d14a941735', 
    'Australian': '4bf58dd8d48988d169941735', 
    'Austrian': '52e81612bcbc57f1066b7a01', 
    'Bangladeshi': '5e179ee74ae8e90006e9a746', 
    'Belgian': '52e81612bcbc57f1066b7a02', 
    'Caribbean': '4bf58dd8d48988d144941735', 
    'Caucasian': '5293a7d53cf9994f4e043a45', 
    'Czech': '52f2ae52bcbc57f1066b8b81', 
    'Dutch': '5744ccdfe4b0c0459246b4d0', 
    'Belarusian': '52e928d0bcbc57f1066b7e97', 
    'Bosnian': '58daa1558bbb0b01f18ec1ee', 
    'Bulgarian': '56aa371be4b08b9a8d5734f3', 
    'Romanian': '52960bac3cf9994f4e043ac4', 
    'Tatar': '52e928d0bcbc57f1066b7e98', 
    'English': '52e81612bcbc57f1066b7a05', 
    'French': '4bf58dd8d48988d10c941735', 
    'German': '4bf58dd8d48988d10d941735', 
    'Greek': '4bf58dd8d48988d10e941735', 
    'Hawaiian': '52e81612bcbc57f1066b79fe', 
    'Hungarian': '52e81612bcbc57f1066b79fa', 
    'Indian': '4bf58dd8d48988d10f941735', 
    'Italian': '4bf58dd8d48988d110941735', 
    'Salvadoran': '5745c7ac498e5d0483112fdb', 
    'Argentinian': '4bf58dd8d48988d107941735', 
    'Brazilian': '4bf58dd8d48988d16b941735', 
    'Colombian': '58daa1558bbb0b01f18ec1f4', 
    'Peruvian': '4eb1bfa43b7b52c0e1adc2e8', 
    'Venezuelan': '56aa371be4b08b9a8d573558', 
    'Moroccan': '4bf58dd8d48988d1c3941735', 
    'Mexican': '4bf58dd8d48988d1c1941735', 
    'Egyptian': '5bae9231bedf3950379f89e1', 
    'Iraqi': '5bae9231bedf3950379f89e7', 
    'Israeli': '56aa371be4b08b9a8d573529', 
    'Kurdish': '5744ccdfe4b0c0459246b4ca', 
    'Lebanese': '58daa1558bbb0b01f18ec1cd', 
    'Persian': '52e81612bcbc57f1066b79f7', 
    'Syrian': '5bae9231bedf3950379f89da', 
    'Yemeni': '5bae9231bedf3950379f89ea', 
    'Pakistani': '52e81612bcbc57f1066b79f8', 
    'Polish': '52e81612bcbc57f1066b7a04', 
    'Portuguese': '4def73e84765ae376e57713a', 
    'Russian': '5293a7563cf9994f4e043a44', 
    'Scandinavian': '4bf58dd8d48988d1c6941735', 
    'Scottish': '5744ccdde4b0c0459246b4a3', 
    'Slovak': '56aa371be4b08b9a8d57355a', 
    'Spanish': '4bf58dd8d48988d150941735', 
    'Sri Lankan': '5413605de4b0ae91d18581a9', 
    'Swiss': '4bf58dd8d48988d158941735', 
    'Turkish': '4f04af1f2fb6e1c99f3db0bb', 
    'Ukranian': '52e928d0bcbc57f1066b7e96'
}
#restaurants.drop(restaurants.index, inplace=True)
#cities = ['Seattle']
#restaurants = pd.DataFrame(columns=['City', 'Cuisine', 'Number of restaurants'])
for city in cities:
    print(city)
    for cuisine_name, category_id in cuisines.items():
        # For each city, retrieve the top restaurants from Foursquare for each cuisine...
        query = requests.get("{url}&near={city}&categoryId={category_id}".format(url=URL, city=city, category_id=category_id))
        if query.status_code == 200:
            number = query.json()
            restaurants = restaurants.append({
                'City': city,
                'Cuisine': cuisine_name,
                'Number of restaurants': number['response']['totalResults']
            }, ignore_index=True)
        
restaurants

Seattle


,City,Cuisine,Number of restaurants
0,New York City,Afghan,13
1,New York City,African,91
2,New York City,American,376
3,New York City,Burmese,9
4,New York City,Cambodian,9
5,New York City,Chinese,350
6,New York City,Filipino,23
7,New York City,Himalayan,12
8,New York City,Indonesian,14
9,New York City,Japanese,340


In [31]:
# Write DataFrame to CSV
#restaurants.to_csv('restaurants.csv', index=False)

In [32]:
restaurants_from_csv = pd.read_csv('restaurants.csv', keep_default_na=False, na_values=[""])
restaurants_from_csv.head()
restaurants = restaurants_from_csv

,City,Cuisine,Number of restaurants
0,New York City,Afghan,13
1,New York City,African,91
2,New York City,American,376
3,New York City,Burmese,9
4,New York City,Cambodian,9


In [21]:
restaurants_total = restaurants.groupby('City').sum().reset_index()
#restaurants['Number of restaurants'][0]
#restaurants_total = restaurants_total[restaurants_total['City','Number of restaurants']]
restaurants_total.head()

,City,Cuisine,Number of restaurants
0,Austin,AfghanAfricanAmericanBurmeseCambodianChineseFi...,1642
1,Chicago,AfghanAfricanAmericanBurmeseCambodianChineseFi...,3207
2,Columbus,AfghanAfricanAmericanBurmeseCambodianChineseFi...,981
3,Dallas,AfghanAfricanAmericanBurmeseCambodianChineseFi...,2215
4,Fort Worth,AfghanAfricanAmericanBurmeseCambodianChineseFi...,1002


In [30]:
restaurants_total.sort_values('Number of restaurants', ascending = False)


,City,Cuisine,Number of restaurants
8,New York City,AfghanAfricanAmericanBurmeseCambodianChineseFi...,5574
7,Los Angeles,AfghanAmericanBurmeseCambodianChineseFilipinoH...,3888
1,Chicago,AfghanAfricanAmericanBurmeseCambodianChineseFi...,3207
5,Houston,AfghanAfricanAmericanBurmeseCambodianChineseFi...,2952
9,Philadelphia,AfghanAfricanAmericanBurmeseCambodianChineseFi...,2458
12,San Diego,AfghanAfricanAmericanBurmeseCambodianChineseFi...,2348
3,Dallas,AfghanAfricanAmericanBurmeseCambodianChineseFi...,2215
13,San Jose,AfghanAfricanAmericanBurmeseCambodianChineseFi...,1732
0,Austin,AfghanAfricanAmericanBurmeseCambodianChineseFi...,1642
10,Phoenix,AfghanAfricanAmericanBurmeseCambodianChineseFi...,1607


In [37]:
del restaurants_total['Cuisine']
restaurants_total.head()


,City,Number of restaurants
0,Austin,1642
1,Chicago,3207
2,Columbus,981
3,Dallas,2215
4,Fort Worth,1002


In [41]:
df_merge_col = pd.merge(restaurants_total, df, on='City')
df_merge_col.sort_values('2019estimate', ascending = False)

,City,Number of restaurants,2019rank,State[c],2019estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
8,New York City,5574,1,New York,8336817,8175133,+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
7,Los Angeles,3888,2,California,3979576,3792621,+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
1,Chicago,3207,3,Illinois,2693976,2695598,−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
5,Houston,2952,4,Texas,2320268,2100263,+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
10,Phoenix,1607,5,Arizona,1680992,1445632,+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W
9,Philadelphia,2458,6,Pennsylvania,1584064,1526006,+3.80%,134.2 sq mi,347.6 km2,"11,683/sq mi","4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W
11,San Antonio,1595,7,Texas,1547253,1327407,+16.56%,461.0 sq mi,"1,194.0 km2","3,238/sq mi","1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W
12,San Diego,2348,8,California,1423851,1307402,+8.91%,325.2 sq mi,842.3 km2,"4,325/sq mi","1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°W
3,Dallas,2215,9,Texas,1343573,1197816,+12.17%,340.9 sq mi,882.9 km2,"3,866/sq mi","1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W
13,San Jose,1732,10,California,1021795,945942,+8.02%,177.5 sq mi,459.7 km2,"5,777/sq mi","2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°W


In [ ]:
# Code to retrieve the venues per city




# Rank the cities with similar number of restaurants

# Check that the GDP is comparable



### Obtaining rankings and grouping by cuisine

In [ ]:
# Group the assessment and group